# Import Libraries

In [1]:
import os
import pandas as pd
import re

# Load Data

In [4]:
path_to_files = './input/'
os.listdir(path_to_files)

['features.csv',
 'patient_notes.csv',
 'sample_submission.csv',
 'test.csv',
 'train.csv']

In [5]:
train_data = pd.read_csv(path_to_files + 'train.csv')
test_data = pd.read_csv(path_to_files + 'test.csv')
sample_submission = pd.read_csv(path_to_files + 'sample_submission.csv')
features = pd.read_csv(path_to_files + 'features.csv')
notes = pd.read_csv(path_to_files + 'patient_notes.csv')

# EDA BLOCK

In [7]:
print(f'Number train samples: {len(train_data)}')
print(f'Number test samples: {len(test_data)}')
print(f'Number samp_subm samples: {len(sample_submission)}')
print(f'Number features samples: {len(features)}')
print(f'Number notes samples: {len(notes)}')

Number train samples: 14300
Number test samples: 5
Number samp_subm samples: 5
Number features samples: 143
Number notes samples: 42146


In [8]:
train_data.head(10)

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']
5,00016_005,0,16,5,[],[]
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']"
7,00016_007,0,16,7,[],[]
8,00016_008,0,16,8,[],[]
9,00016_009,0,16,9,"['palpitations', 'heart beating/pounding']","['26 38', '96 118']"


In [9]:
train_data[(train_data['case_num']==0) & (train_data['feature_num']==11)]['annotation'].value_counts()

['17 yo']           39
['17yo']            17
['17 year old']     12
['17 YO']            8
['17 y/o']           7
['17-year-old']      4
[]                   3
['17 y.o']           2
['17 Y/O']           1
['17 Yo']            1
['17 y.o.']          1
['17']               1
['17 yr old']        1
['17y']              1
['17 years old']     1
['17 yr Old']        1
Name: annotation, dtype: int64

In [10]:
pn_num = 16
notes[notes['pn_num']==pn_num]['pn_history'][pn_num]

'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with "thyroid disease," dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'

In [11]:
train_data[train_data['pn_num']==pn_num]

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']
5,00016_005,0,16,5,[],[]
6,00016_006,0,16,6,"['adderall', 'adderrall', 'adderrall']","['321 329', '404 413', '652 661']"
7,00016_007,0,16,7,[],[]
8,00016_008,0,16,8,[],[]
9,00016_009,0,16,9,"['palpitations', 'heart beating/pounding']","['26 38', '96 118']"


In [12]:
text = notes[notes['pn_num']==pn_num]['pn_history'].values[0]
text

'HPI: 17yo M presents with palpitations. Patient reports 3-4 months of intermittent episodes of "heart beating/pounding out of my chest." 2 days ago during a soccer game had an episode, but this time had chest pressure and felt as if he were going to pass out (did not lose conciousness). Of note patient endorses abusing adderall, primarily to study (1-3 times per week). Before recent soccer game, took adderrall night before and morning of game. Denies shortness of breath, diaphoresis, fevers, chills, headache, fatigue, changes in sleep, changes in vision/hearing, abdominal paun, changes in bowel or urinary habits. \r\nPMHx: none\r\nRx: uses friends adderrall\r\nFHx: mom with "thyroid disease," dad with recent heart attcak\r\nAll: none\r\nImmunizations: up to date\r\nSHx: Freshmen in college. Endorses 3-4 drinks 3 nights / week (on weekends), denies tabacco, endorses trying marijuana. Sexually active with girlfriend x 1 year, uses condoms'

In [13]:
match = re.search('M', text)
print('Start pos: ', match.start())
print('End pos: ', match.end())

Start pos:  10
End pos:  11


In [14]:
match = re.search('17\w+', text)
print('Match:', match.group())
print('Start position:', match.start())
print('End position:', match.end())

Match: 17yo
Start position: 5
End position: 9


In [15]:
df_temp = train_data[train_data['feature_num']==12]
df_temp.head()

,id,case_num,pn_num,feature_num,annotation,location
12,00016_012,0,16,12,['M'],['10 11']
25,00041_012,0,41,12,['M'],['7 8']
38,00046_012,0,46,12,['M'],['24 25']
51,00082_012,0,82,12,['M'],['6 7']
64,00100_012,0,100,12,['male'],['58 62']


In [16]:
df_temp['annotation'].value_counts()

['M']       47
['male']    36
['m']        9
[]           6
['Male']     1
['boy']      1
Name: annotation, dtype: int64

In [17]:
keys = df_temp['annotation'].value_counts().keys()
pattern_list = [key.replace('[', '').replace(']', '').replace("'", '') for key in keys if len(key) > 3]
pattern_list

['M', 'male', 'm', 'Male', 'boy']

In [18]:
data = test_data.copy()

In [21]:
for row in data.index:
    case_num = data.loc[row, 'case_num']
    pn_num = data.loc[row, 'pn_num']
    feature_num = data.loc[row, 'feature_num']
    
    # Create Pattern List For The Feature
    df_temp = train_data[train_data['feature_num']==feature_num].copy()
    keys = df_temp['annotation'].value_counts().keys()
    pattern_list = [key.replace("[", '').replace("]", '').replace("'", '') for key in keys if len(key)>3]
    
    # Extract Patient Note
    text = notes[(notes['pn_num']==pn_num)&(notes['case_num']==case_num)]['pn_history'].values[0]
    text = text.replace('.', ' ')
    pred = ''
    for pattern in pattern_list:
        pattern = pattern.replace('+', 'P').replace('(', ' ').replace(')', ' ')
        text = text.replace('+', 'P')
        if text.find(pattern)>0:
            pred += str(text.find(pattern))+' '+str(text.find(pattern)+len(pattern))+';'
    pred = pred[:-1]
    data.loc[row, 'location'] = pred
    sample_submission.loc[row, 'id'] = data.loc[row, 'id']
    sample_submission.loc[row, 'location'] = pred

In [22]:
sample_submission

,id,location
0,00016_000,696 724
1,00016_001,668 693
2,00016_002,203 217;209 217
3,00016_003,83 91;70 82;70 91
4,00016_004,222 258


In [23]:
sample_submission.to_csv('subm_samp', index=False)